## 11-1 시군구별 인구 단계 구분도 만들기
- 지역별 통꼐치를 색깔 차이로 표현한 지도를 단계 구분도(choroplrth map)이라 부른다
- 단계 구분도를 만들면 인구나 소득과 같은 통계치가 지역별로 어떻게 다른지 쉽게 알 수 있음

## Lab 1 : 시군구별 인구 단계 구분도 만들기
1. 시군구 경계 지도 데이터 준비하기
- 대한민국 시군구별 경계 좌표가 들어 있는 SIG.geojson 파일을 활용
- 행정구역 코드, 지역 이름, 시군구 경계 위도와 경도 좌표
- geojson은 위치 정보를 JSON 포맷으로 저장한 표준 지리 정보 데이터 포맷
- (주)지오서비스에서 공개한 SHP 파일을 오픈 소스 지리정보 시스템 QGIS로 변환하여 생성한 파일
- bit.ly/easypy_111


In [42]:
import json
geo = json.load(open('SIG.geojson', encoding = 'UTF-8'))

- geo는 딕셔너리 자료 구조로 되어 있음
- geo의 properties에 들어 잇는 SIG_CD에 지역을 나타내는 행정구역 코드가 담김
- geometry에는 시군구의 경계를 나타내는 위도, 경도 좌표가 있음

In [43]:
# 행정 구역 코드 출력
geo['features'][0]['properties']

{'SIG_CD': '42110', 'SIG_ENG_NM': 'Chuncheon-si', 'SIG_KOR_NM': '춘천시'}

In [44]:
# 위도 경도 좌표 출력
geo['features'][0]['geometry']

{'type': 'MultiPolygon',
 'coordinates': [[[[127.58508551154958, 38.08062321552708],
    [127.58565575732702, 38.0802009066172],
    [127.58777905808203, 38.080354190085544],
    [127.58890487394689, 38.080881783588694],
    [127.59031267326897, 38.080596307998306],
    [127.59061778023133, 38.08053451807929],
    [127.59137292963024, 38.080476965118685],
    [127.59349548967889, 38.08031540227777],
    [127.594304166838, 38.080225164594665],
    [127.59529626584073, 38.08011430122607],
    [127.59683797157537, 38.079896764445195],
    [127.59703599139921, 38.07914773681181],
    [127.59756973764837, 38.07761547763786],
    [127.59794006383213, 38.07714541130494],
    [127.59816760846913, 38.07685681587356],
    [127.59847648881038, 38.076670284099194],
    [127.59956152109447, 38.076014995491754],
    [127.60022591131202, 38.07568645192285],
    [127.60034758173289, 38.075626138582486],
    [127.6014570759825, 38.07461119532391],
    [127.60157555080873, 38.07455485196505],
    [127.6

1. 시군구별 인구 데이터 준비하기
- Population_SIG.csv : 2021년 시군구별 인구 데이터

In [45]:
import pandas as pd
df_pop = pd.read_csv('Population_SIG.csv')
df_pop.head()

,code,region,pop
0,11,서울특별시,9509458
1,11110,종로구,144683
2,11140,중구,122499
3,11170,용산구,222953
4,11200,성동구,285990


In [46]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    278 non-null    int64 
 1   region  278 non-null    object
 2   pop     278 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.6+ KB


In [47]:
# code를 int64 타입에서 문자 타입으로 변경
df_pop['code'] = df_pop['code'].astype(str)

In [48]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    278 non-null    object
 1   region  278 non-null    object
 2   pop     278 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


2. 단계 구분도 만들기

In [49]:
# folium 패키지 인스톨
!pip install folium

> 

> (1) 배경지도 만들기

In [50]:
import folium

# 배경지도 만들기 - 중심 위경도(35.95, 127.7), 확대 단계 8
# 지도 중심 좌표, 확대 단계, 지도 가로 크기, 세로 크기
folium.Map(location = [35.95, 127.7],
          zoom_start = 8,
          width = '50%',
          height = '50%')

# 배경지도 만들기 - 중심 위경도(35.95, 127.7), 확대 단계 8, 지도 종류 변경
# 지도 종류 - 단계 구분이 잘되는 밝은 색상
map_sig = folium.Map(location = [35.95, 127.7],
                    zoom_start= 8,
                    tiles='cartodbpositron')
map_sig

> (2) 단계 구분도 만들기
> - folium.Choropleth()
    > - geo_data : 지도 데이터
    > - data : 색깔로 표현할 통계 데이터
    > - columns : 통계 데이터의 행정 구역 코드 변수, 색깔로 표현할 변수
    > - key_on : 지도 데이터의 행정구역 코드

# 단계별 구분도 만들기
# 앞서 만든 배경지도 map_sig에 단계구분도를 덧씌움
# map_sig를 실행하면 시군구 경계가 표시된 지도가 출력
# 지도 데이터, 통계 데이터, df_pop 행정구역 코드와 인구, geo 행정구역 코드
folium.Choropleth(geo_data=geo,
                 data = df_pop,
                 columns = ('code', 'pop'),
                 key_on = 'feature.properties.SIG_CD').add_to(map_sig)
map_sig

> 계급 구간 정하기

In [54]:
# 인구를 5가지 게급 구간으로 나눔
# 값을 크기 순으로 나열한 다음 입력한 비율에 해당하는 값인 '분위수'를 구함
bins = list(df_pop["pop"].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
bins

[8867.0, 50539.6, 142382.20000000004, 266978.6, 423107.20000000024, 13565450.0]

> 디자인 수정하기

# 배경지도 만들기 - 중심 위경도(35.95, 127.7), 확대 단계 8, 지도 종류변경
# 지도 중심 위치, 확대 단계, 지도 종류
map_sig = folium.Map(location = [35.95, 127.7],
                    zoom_start = 8,
                    tiles = 'cartodbpositron')
map_sig

# 단계변 구분도 만들기
# 지도 데이터, 통계 데이터, df_pop의 행정구역 코드와 인구, geo 행정구역 코드
# 컬러 맵, 투명도 , 경계선 투명도, 계급 구간 기준 값 / 배경 지도에 추가
folium.Choropleth(geo_data=geo,
                 data=df_pop,
                 columns = ('code', 'pop'),
                 key_on = 'feature.properties.SIG_CD',
                 fill_color = 'YlGnBu',
                 fill_opacity = 1,
                 line_opacity= 0.5,
                 bins = bins).add_to(map_sig)
map_sig

## 11-2 서울시 동별 외국인 인구 단계 구분도 만들기
## Lab 2 :  서울시 동별 외국인 인구 단계 구분도 만들기

1. 서울시 동 경계 지도 데이터 준비하기

In [57]:
# 서울 동 경계 좌표가 들어있는 지도 데이터 불러오기
import json
geo_seoul = json.load(open('EMD_Seoul.geojson', encoding='UTF-8'))

In [58]:
# 행정 구역 코드 출력
geo_seoul['features'][0]['properties']

{'BASE_DATE': '20200630',
 'ADM_DR_CD': '1101053',
 'ADM_DR_NM': '사직동',
 'OBJECTID': '1'}

In [59]:
# 위도, 경도 좌표 출력
geo_seoul['features'][0]['geometry']

{'type': 'MultiPolygon',
 'coordinates': [[[[126.97398562200112, 37.578232670691676],
    [126.97400165856983, 37.578091598158124],
    [126.97401347517625, 37.57797124764524],
    [126.97402588957173, 37.57786305895336],
    [126.97403969022386, 37.57773527607586],
    [126.97405282759235, 37.57763245751229],
    [126.97405724608929, 37.57759228442397],
    [126.97406454937413, 37.5775223449396],
    [126.97406606875553, 37.57751912550699],
    [126.97406875866477, 37.57749510583154],
    [126.97406829700772, 37.5774925030879],
    [126.97406834313652, 37.57749109719524],
    [126.97415332533838, 37.57749252255686],
    [126.97415369692854, 37.57749636934987],
    [126.97414192870245, 37.577607692520026],
    [126.97412902832193, 37.57772060121065],
    [126.97417294290439, 37.57749796032778],
    [126.97417276162724, 37.57749252680757],
    [126.9742642955674, 37.577008320942475],
    [126.97428742236072, 37.576886350575165],
    [126.97430268332849, 37.57679306561384],
    [126.9743

2. 서울실 동별 외국인 인구 데이터 준비하기

In [60]:
# 서울시 동별 외국인 통계 데이터 읽어오기
foreigner = pd.read_csv('Foreigner_EMD_Seoul.csv')
foreigner.head()

,code,region,pop
0,1101053,사직동,418.0
1,1101054,삼청동,112.0
2,1101055,부암동,458.0
3,1101056,평창동,429.0
4,1101057,무악동,102.0


In [61]:
foreigner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3490 entries, 0 to 3489
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    3490 non-null   int64  
 1   region  3490 non-null   object 
 2   pop     3486 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 81.9+ KB


In [62]:
# 행정 구역 코드 타입을 문자로 변경
foreigner['code'] = foreigner['code'].astype(str)

3. 단계별 구분도 만들기

In [63]:
# 외국인 수를 8가지 계급 구간으로 나눔
# 값을 크기 순으로 나열한 다음 입력한 비율에 해당하는 값인 '분위수'를 구함
bins = list(foreigner['pop'].quantile([0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))
bins

[7.0, 98.0, 200.0, 280.0, 386.0, 529.5, 766.0, 1355.5, 26896.0]

# 배경지도 만들기 - 서울 중심의 위경도(35.95, 127.7), 확대 단계 12, 지도 종류 변경
# 서울 중심을 중앙에 배치, 확대 단계 12, 지도 종류
map_seoul = folium.Map(location = [35.95, 127.7],
                      zoom_start = 12,
                      tiles='cartodbpositron')
map_seoul


# 단계별 구분도 만들기
folium.Choropleth(
geo_data=geo_seoul, # 서울 좌표
data = foreigner, # 통계 데이터
columns = ('code', 'pop'), # 통계 데이터에서 행정구역 코드(code), 인구
key_on = 'feature.properties.ADM_DR_CD', # geo_seoul의 행정구역 코드
fill_color = 'Blues', # 컬러맵
nan_fill_color = 'White', # 결측치 색깔
fill_opacity = 1, # 투명도
line_opacity = 0.5, # 경게선 투명도
bins = bins).add_to(map_seoul) # 계급 구간 기준값 / 서울 배경지도에 추가
map_seoul


4. 구 경계선 추가하기

In [71]:
# 구 경계 좌표를 담은 파일 읽어오기
geo_seoul_sig = json.load(open('SIG_Seoul.geojson', encoding='UTF-8'))

# 서울 구 라인 추가
# 지도 데이터, 투명도, 선 두께 / 배경지도에 추가
folium.Choropleth(geo_data=geo_seoul_sig,
                 fill_opacity=0,
                 line_weight = 4).add_to(map_seoul)
map_seoul

- html로 저장하기

In [68]:
# map을 html 파일로 저장, 웹브라우저로 확인 가능
map_seoul.save('map_seoul.html')

In [69]:
# 웹브라우저 패키지의 open_new()를 이용하여 HTML 파일을 열기
import webbrowser
webbrowser.open_new('map_seoul.html')

True

- 다양한 지도 만들기
    > python-visualization.github.io/folium


## 14-1 가설 검증이란 ?
### 기술통계와 추론 통계
- 기술통계 (Descriptive Statistics) : 데이터를 요약하여 설명하는 통계분석 기법
> (예) 사람들이 받는 월급을 집계하여 전체 월급 평균을 구함
- 추론 통계 (inferential Statistics) : 단순히 숫자를 요약하는 것을 넘어 어떤 값이 발생할 확률을 계산하는 통계분석 기법
> (예) 데이터에서 성별에 따른 월급 차이가 발생한 경우, 이런 차이가 우연히 발생할 확률 계산 (통계적 유의성)
- 일반적으로 통계분석을 수행했다고 한 것은 추론 통계를 이용하여 가설 검증을 수행한 것을 의미함
- 기술 통계 분석에서 집단간에 차이가 있더라도 우연에 의한 차이일 수 있음.
- 유의 확률을 계산하는 통계적 가설 검증 절차를 거쳐야 함

### 통계적 가설 검증
- 유의 확률을 이용하여 가설을 검증하는 방법
- 유의 확률은 실제로 집단간 차이가 없는데 우영히 차이가 있는 데이터가 추출될 확률을 의미
> 유의확률이 크다 : 집단간의 차이가 통계적으로 유의하지 않다. 우연히 차이가 관찰될 가능성이 크다 유의확률이 작다 : 집단간 차이가 통계적으로
유의하다. 우연히 차이가 관찰될 가능성이 작다
- t 검정 : 두 집단의 평균에 차이가 있는지 검정하는 방법
- 상관관계 : 두 변수가 관련이 있는지 검정하는 방법

## 14-2 t 검증 - 두 집단의 평균 비교하기
- t 검정 : 두 집단의 평균에 차이가 있는지 검정하는 방법

## Lab 1 : compact 자동차와 sub 자동차의 도시 연비 t
검증


In [77]:
import pandas as pd

In [78]:
mpg = pd.read_csv('mpg.csv')

In [80]:
# 기술 통계 분석
mpg.query('category in ["compact", "suv"]')\
    .groupby('category', as_index=False)\
    .agg(n = ('category', 'count'),
        mean = ('cty', 'mean'))

,category,n,mean
0,compact,47,20.12766
1,suv,62,13.50000


- compact 자동차가 suv에 비해 도시 연비가 높다(의미있는 차이인가?)
- t 검증을 통해 확인

In [81]:
compact = mpg.query('category == "compact"')['cty']
compact.head()

0    18
1    21
2    20
3    21
4    16
Name: cty, dtype: int64

In [82]:
suv = mpg.query('category == "suv"')['cty']
suv.head()

18    14
19    11
20    14
21    13
22    12
Name: cty, dtype: int64

In [83]:
# t - test
from scipy import stats
stats.ttest_ind(compact, suv, equal_var=True)

Ttest_indResult(statistic=11.917282584324107, pvalue=2.3909550904711282e-21)

- 일반적으로 p-value가 5%(0.05) 미만이면 '집단 간 차이가 통계적으로 유의하다'라고 해석
- 우연히 관찰될 확률이 5%보다 작으면, 이 차이를 우연이라고 보기 어렵다고 결론을 내림

> p-value가 2.3909550904711282e-21로 0.05(5%)보다 작으면 평균 도시 차이가 통계적으로 유의미하다고 판단할 수 있음

## Lab 2 : 일반 휘발유와 고급 휘발유 도시 연비 t 검증

In [84]:
# 기술 통계 분석
mpg.query('fl in ["r", "p"]')\
    .groupby('fl', as_index=False)\
    .agg(n = ('fl', 'count'),
        mean = ('cty', 'mean'))

,fl,n,mean
0,p,52,17.365385
1,r,168,16.738095


> 고급 휘발유의 도시 연비가 다소 높다.(실제 유의미한가?)

In [85]:
regular = mpg.query('fl == "r"')['cty']
regular.head()

18    14
20    14
21    13
22    12
28    14
Name: cty, dtype: int64

In [86]:
premium = mpg.query('fl == "p"')['cty']
premium.head()

0    18
1    21
2    20
3    21
4    16
Name: cty, dtype: int64

In [87]:
# t-test
from scipy import stats
stats.ttest_ind(regular, premium, equal_var=True)

Ttest_indResult(statistic=-1.066182514588919, pvalue=0.28752051088667036)

> t value가 0.28752051088667036로 0.05(5%)보다 크므로, 평균 도시 차이가 통계적으로 유의미하지 않다
고급 휴발유와 일반 휴발유의 도시 연비 차이는 우연히 발생할 가능성이 크다

## 14-3 상관분석 - 두 변수의 관계 분석하기
- 상관분석(correation ananlysis) : 두 연속 변수가 서로 관련이 있는지 검정하는 통계 분석 기법
- 상관계수(correation coefficient)를 보면 두 변수가 얼마나 관계가 있는지 정도를 파악할 수 있음

### Lab 3 : 실업자 수와 개인 소비 지출의 상관관계
- economic 데이터를 활용하여 unemploy(실업자 수)와 pce(개인 소비 지출)간의 상관 관계를 알아보자

In [88]:
# economic 데이터 불러오기
economics = pd.read_csv('economics.csv')

# 상관 행렬 만들기
economics[['unemploy', 'pce']].corr()

,unemploy,pce
unemploy,1.000000,0.614518
pce,0.614518,1.000000


- 상관계수는 -1 ~ 1 값을 갖는다
- 0.4 이상 또는 -0.5 이하인 경우 양의 또는 음의 관련성이 있다라고 해석
- unemploy(실업자 수)와 pce(개인 소비 지출)는 상관계수가 0.614518로 양의 관련성이 있음
- 실업자 수와 소비 지출은 한 변수가 증가하면 다른 변수가 증가하는 비례 관계임

## 유의 확률 구하기

In [89]:
# 상관 분석
stats.pearsonr(economics['unemploy'], economics['pce'])

PearsonRResult(statistic=0.6145176141932082, pvalue=6.773527303291316e-61)

- 첫 번째 값은 상관계수, 두 번째 값은 유의 확률을 나타냄
- 유의 확률이 0.05 미만으로 실업자 수와 소비 지출의 상관관계가 통계적으로 유의하다고 결론

## Lab 4 : 상관행렬 히트맵 만들기
- 상관행렬 만들기
> mtcars: 자동차 32종의 11개 변수로 구성된 데이터 
> - bit.ly/easypy_141|

In [90]:
mtcars = pd.read_csv('mtcars.csv')
mtcars.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [91]:
# 상관 행렬 만들기
car_cor = mtcars.corr()

# 소숫점 둘째 자리까지 반올림
car_cor = round(car_cor, 2)
car_cor

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
mpg,1.00,-0.85,-0.85,-0.78,0.68,-0.87,0.42,0.66,0.60,0.48,-0.55
cyl,-0.85,1.00,0.90,0.83,-0.70,0.78,-0.59,-0.81,-0.52,-0.49,0.53
disp,-0.85,0.90,1.00,0.79,-0.71,0.89,-0.43,-0.71,-0.59,-0.56,0.39
hp,-0.78,0.83,0.79,1.00,-0.45,0.66,-0.71,-0.72,-0.24,-0.13,0.75
drat,0.68,-0.70,-0.71,-0.45,1.00,-0.71,0.09,0.44,0.71,0.70,-0.09
wt,-0.87,0.78,0.89,0.66,-0.71,1.00,-0.17,-0.55,-0.69,-0.58,0.43
qsec,0.42,-0.59,-0.43,-0.71,0.09,-0.17,1.00,0.74,-0.23,-0.21,-0.66
vs,0.66,-0.81,-0.71,-0.72,0.44,-0.55,0.74,1.00,0.17,0.21,-0.57
am,0.60,-0.52,-0.59,-0.24,0.71,-0.69,-0.23,0.17,1.00,0.79,0.06
gear,0.48,-0.49,-0.56,-0.13,0.70,-0.58,-0.21,0.21,0.79,1.00,0.27


- mpg(연비)와 cly(실린더 수) 상관계수는 -0.85이므로 연비가 높을수록 실린더 수가 적은 경향이 있음
- cly(실린더 수)와 wt(무게)의 상관게수가 0.78이므로, 실린더 수가 많을 수록 자동차가 무거운 경향이 있음

## 히트맵 만들기

In [92]:
# 해상도 결정, 가로와 세로 크기 설정
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi' : '120',
                    'figure.figsize' : [7.5, 5.5]})

# 상관계수 표시, 컬러맵
import seaborn as sns
sns.heatmap(car_cor,
           annot = True,
           cmap = 'RdBu')

- 히트맵은 상관계수가 클수록 상자 색깔을 진하게 표현하고,
- 상관게수가 양수이면 파란색, 음수이면 빨간색 계열로 표현함 

### 대각 행렬 제거하기

- mask 만들기

In [94]:
# mask 만들기
# 상관행렬의 행과 열 수만큼 0으로 채움
import numpy as np
mask = np.zeros_like(car_cor)
mask

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [95]:
# 오른쪽 위 대각 행렬을 1로 바꾸기
mask[np.triu_indices_from(mask)] = 1
mask

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

- 히트맵에 적용

# 상관계수 표시, 컬러맵, mask 적용
sns.heatmap(data = car_cor,
           annot = True,
           cmap = 'RdBu',
           mask = mask)

- 빈행과 열 제거

mask_new = mask[1:, :-1]              # mask의 첫 번째 행, 마지막 열 제거
cor_new = car_cor.iloc[1:, :-1]       # 상관행렬 첫 번째 행, 마지막 열 제거

# 히트맵 만들기
# 상관계수 표시, 컬러맵, mask 적용
sns.heatmap(data= cor_new,
           annot = True,
           cmap = 'RdBu',
           mask = mask_new)           

sns.heatmap(data=cor_new,
           annot = True,
           cmap = 'RdBu',
           mask = mask_new,
           linewidths = 0.5,
           vmax = 1,
           vmin = -1,
           cbar_kws = {"shrink" : 0.5})

# 가장 진한 파란색 최대값
# 가장 진한 빨간색 최소값
# 범례 크기 줄이기